In [1]:
import numpy as np

In [46]:
class Game:
    def __init__(self, nb_rows=6, nb_columns=7, nb_players=2):
        self.nb_rows = nb_rows
        self.nb_columns = nb_columns
        self.nb_players = nb_players
        self.grid = np.zeros((nb_rows, nb_columns))
        self.turn = 0
        
    def is_legal(self, column):
        col_values = self.grid[:, column]
        non_zeros, = np.where(col_values == 0)
        return not (len(non_zeros) == 0)
    
    def set_token(self, column):
        try:
            assert self.is_legal(column)
        except:
            raise Exception("The move is not legal. Column {} is already filled".format(column))
        
        row_index = np.where(self.grid[:, column] == 0)[0][0]
        player_token = 1
        if self.turn == 1:
            player_token = -1
            
        self.grid[row_index][column] = player_token
        
        self.turn = (self.turn + 1) % self.nb_players
        
    def won(self, player_id):
        token_player = 1
        if player_id == 1:
            token_player = -1
            
        for row in range(3, self.nb_rows):
            """pb il ne faut pas commencer à 3"""
            for column in range(3, self.nb_columns):
                current_token = self.grid[row][column]
                if current_token == token_player:
                    a = True
                    for k in range(1, 4):
                        if grid[row - k][column] != current_token:
                            a = False
                    
                    if a:
                        return True
                    
                    a = True
                    for k in range(1, 4):
                        if grid[row][column - k] != current_token:
                            a = False
                            
                    if a:
                        return True
                            
                    a = True
                    for k in range(1, 4):
                        if grid[row - k][column - k] != current_token:
                            a = False
                            
                    if a:
                        return True
                    
        return False

In [47]:
game = Game()

In [48]:
game.set_token(0)
game.set_token(1)

game.set_token(0)
game.set_token(1)

game.set_token(0)
game.set_token(1)

game.set_token(0)
game.set_token(1)

In [49]:
game.won(0)

yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah


False